In [1]:
! pip install -r ../requirements.txt

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool
import fasttext
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import NaiveBayesClassifier
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report 
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import MultinomialNB
import gc

In [2]:
def nb_classification(train, label, vectorizer='bow'):

    train = train.apply(' '.join)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(train, label, test_size=0.3, random_state=42)

    if vectorizer == 'bow':
        vec = CountVectorizer()
    elif vectorizer == 'tfidf':
        vec = TfidfVectorizer()

    X_train_vec = vec.fit_transform(X_train)
    X_test_vec = vec.transform(X_test)

    # Initialize the MultinomialNB
    nb = MultinomialNB()

    # Fit the model to the training data
    nb.fit(X_train_vec, y_train)

    # Predict the labels of the test set
    y_pred = nb.predict(X_test_vec)

    # Generate classification report
    report = classification_report(y_test, y_pred)
    return report

    

### Tokenized

In [3]:
df = pd.read_pickle('/work/pkl_files/tokenized.pkl')

In [4]:
df.head()

,title,tag,artist,year,lyrics,id,lyrics_word_count
0,Killa Cam,rap,Cam'ron,2004,"[killa, cam, killa, cam, cam, killa, cam, kill...",1,762
1,Can I Live,rap,JAY-Z,1996,"[yeah, hah, yeah, rocafella, invite, somethin,...",3,548
2,Forgive Me Father,rap,Fabolous,2003,"[maybe, cause, im, eatin, bastards, fiend, gru...",4,574
3,Down and Out,rap,Cam'ron,2004,"[ugh, killa, baby, kanye, 1970s, heron, flow, ...",5,760
4,Fly In,rap,Lil Wayne,2005,"[ask, young, boy, gon, second, time, around, g...",6,432


In [5]:
#bow
bow_report = nb_classification(df['lyrics'], df['tag'], vectorizer='bow')
print(bow_report)

              precision    recall  f1-score   support

     country       0.33      0.18      0.23     25477
         pop       0.65      0.49      0.56    430965
         rap       0.83      0.79      0.81    298959
          rb       0.24      0.35      0.29     47343
        rock       0.40      0.62      0.48    191812

    accuracy                           0.59    994556
   macro avg       0.49      0.49      0.48    994556
weighted avg       0.63      0.59      0.60    994556



In [6]:
#tfidf
tfidf_report = nb_classification(df['lyrics'], df['tag'], vectorizer='tfidf')
print(tfidf_report)

              precision    recall  f1-score   support

     country       0.00      0.00      0.00     25477
         pop       0.58      0.91      0.71    430965
         rap       0.80      0.83      0.82    298959
          rb       0.13      0.00      0.00     47343
        rock       0.69      0.05      0.09    191812

    accuracy                           0.65    994556
   macro avg       0.44      0.36      0.32    994556
weighted avg       0.63      0.65      0.57    994556



### Stemmed

In [4]:
df = pd.read_pickle('/work/pkl_files/stemmed.pkl')

In [5]:
df.head()

,title,tag,artist,year,lyrics,id,lyrics_word_count,stemmed_lyrics
0,Killa Cam,rap,Cam'ron,2004,"[killa, cam, killa, cam, cam, killa, cam, kill...",1,762,"[killa, cam, killa, cam, cam, killa, cam, kill..."
1,Can I Live,rap,JAY-Z,1996,"[yeah, hah, yeah, rocafella, invite, somethin,...",3,548,"[yeah, hah, yeah, rocafella, invit, somethin, ..."
2,Forgive Me Father,rap,Fabolous,2003,"[maybe, cause, im, eatin, bastards, fiend, gru...",4,574,"[mayb, caus, im, eatin, bastard, fiend, grub, ..."
3,Down and Out,rap,Cam'ron,2004,"[ugh, killa, baby, kanye, 1970s, heron, flow, ...",5,760,"[ugh, killa, babi, kany, 1970s, heron, flow, h..."
4,Fly In,rap,Lil Wayne,2005,"[ask, young, boy, gon, second, time, around, g...",6,432,"[ask, young, boy, gon, second, time, around, g..."


In [11]:
#bow
bow_report = nb_classification(df['stemmed_lyrics'], df['tag'], vectorizer='bow')
print(bow_report)


              precision    recall  f1-score   support

     country       0.32      0.18      0.23     25477
         pop       0.65      0.48      0.55    430965
         rap       0.83      0.79      0.81    298959
          rb       0.24      0.35      0.28     47343
        rock       0.39      0.63      0.48    191812

    accuracy                           0.59    994556
   macro avg       0.48      0.49      0.47    994556
weighted avg       0.63      0.59      0.59    994556



In [12]:
#tfidf
tfidf_report = nb_classification(df['stemmed_lyrics'], df['tag'], vectorizer='tfidf')
print(tfidf_report)


              precision    recall  f1-score   support

     country       0.00      0.00      0.00     25477
         pop       0.58      0.91      0.71    430965
         rap       0.80      0.83      0.81    298959
          rb       0.14      0.00      0.00     47343
        rock       0.70      0.05      0.09    191812

    accuracy                           0.65    994556
   macro avg       0.44      0.36      0.32    994556
weighted avg       0.63      0.65      0.57    994556



### Lemmatized

In [4]:
df = pd.read_pickle('/work/pkl_files/lemmatized.pkl')
gc.collect()

0

In [5]:
df.head()

,title,tag,artist,year,lyrics,id,lyrics_word_count,lemmatized_lyrics
0,Killa Cam,rap,Cam'ron,2004,"[killa, cam, killa, cam, cam, killa, cam, kill...",1,762,"[killa, cam, killa, cam, cam, killa, cam, kill..."
1,Can I Live,rap,JAY-Z,1996,"[yeah, hah, yeah, rocafella, invite, somethin,...",3,548,"[yeah, hah, yeah, rocafella, invite, somethin,..."
2,Forgive Me Father,rap,Fabolous,2003,"[maybe, cause, im, eatin, bastards, fiend, gru...",4,574,"[maybe, cause, im, eatin, bastard, fiend, grub..."
3,Down and Out,rap,Cam'ron,2004,"[ugh, killa, baby, kanye, 1970s, heron, flow, ...",5,760,"[ugh, killa, baby, kanye, 1970s, heron, flow, ..."
4,Fly In,rap,Lil Wayne,2005,"[ask, young, boy, gon, second, time, around, g...",6,432,"[ask, young, boy, gon, second, time, around, g..."


In [10]:
#bow
bow_report = nb_classification(df['lemmatized_lyrics'], df['tag'], vectorizer='bow')
print(bow_report)


              precision    recall  f1-score   support

     country       0.33      0.17      0.23     25477
         pop       0.65      0.49      0.56    430965
         rap       0.83      0.79      0.81    298959
          rb       0.24      0.34      0.28     47343
        rock       0.40      0.62      0.48    191812

    accuracy                           0.59    994556
   macro avg       0.49      0.48      0.47    994556
weighted avg       0.63      0.59      0.60    994556



In [ ]:
#tfidf
tfidf_report = nb_classification(df['lemmatized_lyrics'], df['tag'], vectorizer='tfidf')
print(tfidf_report)